# KNeighborsClassifier Demo
This demo illustrates how SIMBSIG can be used for Nearest Neighbors classification queries, and how the use compares to scikit-learn. 

In [ ]:
!pip install simbsig

In [ ]:
from simbsig.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier as KNeighborsClassifier_sk
import h5py as h5py
import numpy as np

## Set Parameters

In [ ]:
n_train_samples = 10
n_classes = 3
n_query_samples = 5
n_dim = 2

## Create Toy Data
### numpy arrays

In [ ]:
# numpy arrays
np.random.seed(42)
X_train = np.random.uniform(low=-5, high=5, size=(n_train_samples, n_dim))
X_query = np.random.uniform(low=-5, high=5, size=(n_query_samples, n_dim))
y_train = np.random.randint(low=0, high=n_classes, size=(n_train_samples))

### hdf5 files
If you work from your computer, you can execute the statement below: Of course you can also store the data in a directory more suitable to you.

In [ ]:
# when working from your computer, your disk can be used to save and read data
import os
dataset_path = os.path.dirname(os.path.realpath("__file__"))

In [ ]:
# hdf5 files using h5py
train_file = f'train_KNeighborsClassifier.hdf5'
query_file = f'query_KNeighborsClassifier.hdf5'

with h5py.File(os.path.join(dataset_path, f"{train_file}"), 'w') as f:
    f.create_dataset("X", data=X_train)
    f.create_dataset("y",data=y_train)

with h5py.File(os.path.join(dataset_path, f"{query_file}"), 'w') as f:
    f.create_dataset("X", data=X_query)

## Scikit-Learn


In [ ]:
nn_sk = KNeighborsClassifier_sk(n_neighbors=2)

nn_sk.fit(X_train, y_train)
nn_sk.predict(X_query)

## SIMBSIG
### Using numpy arrays and CPU only
SIMBSIG can be used very similar to scikit-learn. In an existing workflow using scikit-learn, which may be on the verge of exceeding runtime or memory requirements, this allows a seamless transition to SIMBSIG.


In [ ]:
nn = KNeighborsClassifier(n_neighbors=2, verbose=False)

nn.fit(X_train, y_train)
nn.predict(X_query)

### Using hdf5 files and CPU only
If saving the entire data at once in the computer memory using numpy arrays is not reasonable anymore, the hdf5 file format can help. SIMBSIG can use data in hdf5 files, by setting the `mode` argument to `cpu`

In [ ]:
nn_hdf5 = KNeighborsClassifier(n_neighbors=2, mode='hdf5', verbose=False)

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

nn_hdf5.fit(train_data)
pred = nn_hdf5.predict(query_data)

# close hdf5 file
train_data.close()
query_data.close()

pred

## Using GPU acceleration
If data gets big, the execution time becomes an issue. SIMBSIG features GPU acceleration, by setting the `device` argument to `gpu`. This works with both inputs - numpy arrays and hdf5 files.
<font color='red'> Be aware: This only runs without error if a GPU is available! </font>

In [ ]:
nn_hdf5 = KNeighborsClassifier(n_neighbors=2, mode='hdf5', device='gpu', verbose=False)

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

nn_hdf5.fit(train_data)
pred = nn_hdf5.predict(query_data)

# close hdf5 file
train_data.close()
query_data.close()

pred